In [8]:
import pandas as pd
from scipy import stats
from matplotlib import cm, colors
from mpl_toolkits.axes_grid1 import ImageGrid
import json
import pickle
import csv
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import glob, os
import random

import tensorflow as tf
from tensorflow import keras

physical_devices = tf.config.list_physical_devices('GPU')
print(physical_devices)
try:
    for kgpu in range(len(physical_devices)):
        tf.config.experimental.set_memory_growth(physical_devices[kgpu], True)
except:
    # Invalid device or cannot modify virtual devices once initialized.
    pass
#import psyplot.plotting.mapplot as mplot
import psyplot.project as psy
import matplotlib as mpl
%matplotlib inline
%config InlineBackend.close_figures = False

2023-05-25 00:57:01.372385: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-25 00:57:02.073817: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


[]


2023-05-25 00:57:02.936254: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [12]:
# Estimate R2 based on Mooers et al. diagnostics
def estimate_R2_out(y_true,y_pred,nlev,nday,nlat):
    Dimension=y_true.shape
    R2 = np.zeros([nlev,nlat])
    SSE = np.zeros([nlev,nlat])
    SVAR = np.zeros([nlev,nlat])
    for i in range(nlat):
        for z in range(nlev):
            y_true_temp = y_true[z,:,i]
            y_pred_temp = y_pred[z,:,i]
            y_true_temp.shape
            SSE[z,i] = np.sum((y_true_temp-y_pred_temp)**2.0)
            SVAR[z,i] = np.sum((y_true_temp-np.mean(y_true_temp))**2.0)
            R2[z,i] = 1-(SSE[z,i]/SVAR[z,i])
        
    return SSE,SVAR,R2

In [ ]:
import os
import netCDF4 as nc
folder_path = "/pscratch/sd/h/heroplr/R2_analysis/regrid"  # Replace with the actual folder path
out_folder = "/pscratch/sd/h/heroplr/R2_analysis/regrid/Figures"

for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    
    if os.path.isfile(file_path):
        # Perform operations on the file
        print(file_path)  # Example: Print the file path
        TR2_out_file = xr.open_dataset(file_path)
        TR_2 = TR2_out_file.TR2.values
        QR_2 = TR2_out_file.QR2.values
        P_2 = TR2_out_file.P.values
        lon_C = TR2_out_file.lon.values
        lat_C = TR2_out_file.lat.values
        T_tend_true_avg_2 = TR2_out_file.T_tend_true_avg.values
        T_pred_true_avg_2 = TR2_out_file.T_pred_true_avg.values
        Q_tend_true_avg_2 = TR2_out_file.Q_tend_true_avg.values
        Q_pred_true_avg_2 = TR2_out_file.Q_pred_true_avg.values
        TR2_reshape_lat = np.nanmean(TR_2,axis=2)
        QR2_reshape_lat = np.mean(QR_2,axis=2)
        T_tend_true_avg_2_reshape_lat = np.nanmean(T_tend_true_avg_2,axis=3)
        T_pred_true_avg_2_reshape_lat = np.mean(T_pred_true_avg_2,axis=3)
        Q_tend_true_avg_2_reshape_lat = np.nanmean(Q_tend_true_avg_2,axis=3)
        Q_pred_true_avg_2_reshape_lat = np.mean(Q_pred_true_avg_2,axis=3)
        P2_reshape_lat = np.mean(P_2,axis=2)
        TSSE,TSVAR,TR2v2 = estimate_R2_out(T_tend_true_avg_2_reshape_lat,T_pred_true_avg_2_reshape_lat,60,31,180)
        QSSE,QSVAR,QR2v2 = estimate_R2_out(Q_tend_true_avg_2_reshape_lat,Q_pred_true_avg_2_reshape_lat,60,31,180)
        XpC, YpC = np.meshgrid(lat_C,P2_reshape_lat[:,0])
        
        #fig, ax = plt.subplots(2,2, figsize=(15,15))
        fig, ax = plt.subplots(1,2, figsize=(15,5))
        fz = 20
        contour_plot = ax[0].pcolor(XpC, P2_reshape_lat/100, TR2v2, cmap = 'Blues', vmin = 0, vmax = 1.0)
        ax[0].contour(XpC, P2_reshape_lat/100 , TR2v2, [0.7], colors='pink', linewidths=[4])
        ax[0].contour(XpC, P2_reshape_lat/100, TR2v2, [0.9], colors='orange', linewidths=[4])
        ax[0].set_title("(a) Heating", fontsize = fz)
        ax[0].set_ylim(ax[0].get_ylim()[::-1])
        ax[0].set_ylabel("Pressure (hPa)", fontsize = fz)
        ax[0].locator_params(nbins=8)
        ax[0].tick_params(axis='x', labelsize=fz*0.9)
        ax[0].tick_params(axis='y', labelsize=fz*0.9)

        ax[1].pcolor(XpC, P2_reshape_lat/100, QR2v2,  cmap = 'Blues', vmin = 0, vmax = 1.0)
        ax[1].contour(XpC, P2_reshape_lat/100, QR2v2,  [0.7], colors='pink', linewidths=[4])
        ax[1].contour(XpC, P2_reshape_lat/100, QR2v2,  [0.9], colors='orange', linewidths=[4])
        ax[1].set_title("(b) Moistening", fontsize = fz)
        ax[1].set_ylim(ax[1].get_ylim()[::-1])
        ax[1].locator_params(nbins=8)
        ax[1].set_yticks([])
        ax[1].tick_params(axis='x', labelsize=fz*0.9)

        #ax[1].yaxis.set_label_coords(-1.38,-0.09)

        fig.subplots_adjust(right=0.8)
        cbar_ax = fig.add_axes([0.82, 0.12, 0.02, 0.76])
        fig.colorbar(contour_plot, label="Skill Score "+r'$\left(\mathrm{R^{2}}\right)$', cax=cbar_ax)
        #plt.suptitle("Trained DNN Skill for Vertically Resolved Tendencies", y = 0.97)
        plt.subplots_adjust(hspace=0.13)
        figure_filename = file_path[-30:-8]+'.pdf'
        plt.savefig(figure_filename)
        
        
        

/pscratch/sd/h/heroplr/R2_analysis/regrid/step2_lot-37_trial_0135.best_aave.nc


/tmp/ipykernel_103632/2418165556.py:14: RuntimeWarning: divide by zero encountered in double_scalars
  R2[z,i] = 1-(SSE[z,i]/SVAR[z,i])
/tmp/ipykernel_103632/3271378258.py:36: UserWarning: The input coordinates to pcolor are interpreted as cell centers, but are not monotonically increasing or decreasing. This may lead to incorrectly calculated cell edges, in which case, please supply explicit cell edges to pcolor.
  contour_plot = ax[0].pcolor(XpC, P2_reshape_lat/100, TR2v2, cmap = 'Blues', vmin = 0, vmax = 1.0)
/tmp/ipykernel_103632/3271378258.py:46: UserWarning: The input coordinates to pcolor are interpreted as cell centers, but are not monotonically increasing or decreasing. This may lead to incorrectly calculated cell edges, in which case, please supply explicit cell edges to pcolor.
  ax[1].pcolor(XpC, P2_reshape_lat/100, QR2v2,  cmap = 'Blues', vmin = 0, vmax = 1.0)


/pscratch/sd/h/heroplr/R2_analysis/regrid/step2_lot-42_trial_0110.best_aave.nc


/tmp/ipykernel_103632/2418165556.py:14: RuntimeWarning: divide by zero encountered in double_scalars
  R2[z,i] = 1-(SSE[z,i]/SVAR[z,i])
/tmp/ipykernel_103632/3271378258.py:36: UserWarning: The input coordinates to pcolor are interpreted as cell centers, but are not monotonically increasing or decreasing. This may lead to incorrectly calculated cell edges, in which case, please supply explicit cell edges to pcolor.
  contour_plot = ax[0].pcolor(XpC, P2_reshape_lat/100, TR2v2, cmap = 'Blues', vmin = 0, vmax = 1.0)
/tmp/ipykernel_103632/3271378258.py:46: UserWarning: The input coordinates to pcolor are interpreted as cell centers, but are not monotonically increasing or decreasing. This may lead to incorrectly calculated cell edges, in which case, please supply explicit cell edges to pcolor.
  ax[1].pcolor(XpC, P2_reshape_lat/100, QR2v2,  cmap = 'Blues', vmin = 0, vmax = 1.0)


/pscratch/sd/h/heroplr/R2_analysis/regrid/step2_lot-35_trial_0070.last_aave.nc


...

/tmp/ipykernel_103632/2418165556.py:14: RuntimeWarning: divide by zero encountered in double_scalars
  R2[z,i] = 1-(SSE[z,i]/SVAR[z,i])
/tmp/ipykernel_103632/3271378258.py:36: UserWarning: The input coordinates to pcolor are interpreted as cell centers, but are not monotonically increasing or decreasing. This may lead to incorrectly calculated cell edges, in which case, please supply explicit cell edges to pcolor.
  contour_plot = ax[0].pcolor(XpC, P2_reshape_lat/100, TR2v2, cmap = 'Blues', vmin = 0, vmax = 1.0)
/tmp/ipykernel_103632/3271378258.py:46: UserWarning: The input coordinates to pcolor are interpreted as cell centers, but are not monotonically increasing or decreasing. This may lead to incorrectly calculated cell edges, in which case, please supply explicit cell edges to pcolor.
  ax[1].pcolor(XpC, P2_reshape_lat/100, QR2v2,  cmap = 'Blues', vmin = 0, vmax = 1.0)


/pscratch/sd/h/heroplr/R2_analysis/regrid/step2_lot-45_trial_0048.last_aave.nc


/tmp/ipykernel_103632/2418165556.py:14: RuntimeWarning: divide by zero encountered in double_scalars
  R2[z,i] = 1-(SSE[z,i]/SVAR[z,i])
/tmp/ipykernel_103632/3271378258.py:36: UserWarning: The input coordinates to pcolor are interpreted as cell centers, but are not monotonically increasing or decreasing. This may lead to incorrectly calculated cell edges, in which case, please supply explicit cell edges to pcolor.
  contour_plot = ax[0].pcolor(XpC, P2_reshape_lat/100, TR2v2, cmap = 'Blues', vmin = 0, vmax = 1.0)
/tmp/ipykernel_103632/3271378258.py:46: UserWarning: The input coordinates to pcolor are interpreted as cell centers, but are not monotonically increasing or decreasing. This may lead to incorrectly calculated cell edges, in which case, please supply explicit cell edges to pcolor.
  ax[1].pcolor(XpC, P2_reshape_lat/100, QR2v2,  cmap = 'Blues', vmin = 0, vmax = 1.0)


/pscratch/sd/h/heroplr/R2_analysis/regrid/step2_lot-41_trial_0103.best_aave.nc


/tmp/ipykernel_103632/2418165556.py:14: RuntimeWarning: divide by zero encountered in double_scalars
  R2[z,i] = 1-(SSE[z,i]/SVAR[z,i])
/tmp/ipykernel_103632/3271378258.py:36: UserWarning: The input coordinates to pcolor are interpreted as cell centers, but are not monotonically increasing or decreasing. This may lead to incorrectly calculated cell edges, in which case, please supply explicit cell edges to pcolor.
  contour_plot = ax[0].pcolor(XpC, P2_reshape_lat/100, TR2v2, cmap = 'Blues', vmin = 0, vmax = 1.0)
/tmp/ipykernel_103632/3271378258.py:46: UserWarning: The input coordinates to pcolor are interpreted as cell centers, but are not monotonically increasing or decreasing. This may lead to incorrectly calculated cell edges, in which case, please supply explicit cell edges to pcolor.
  ax[1].pcolor(XpC, P2_reshape_lat/100, QR2v2,  cmap = 'Blues', vmin = 0, vmax = 1.0)
